In [4]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from datasets import load_dataset
import pandas as pd

In [2]:
# # Load dataset in streaming mode
# ds_stream = load_dataset("abisee/cnn_dailymail", "3.0.0", split='train', streaming=True)

# # Use `take` to load only the first 100,000 rows
# ds_subset = ds_stream.take(100000)

# # Convert to a list to get it into memory
# ds_list = list(ds_subset)

# print(f"Number of rows loaded: {len(ds_list)}")

Number of rows loaded: 100000


In [6]:
import json

# Save to a .jsonl file
with open("subset_cnn_dailymail.jsonl", "w") as f:
    for item in ds_list:
        json.dump(item, f)
        f.write("\n")

# # Load the .jsonl file
# with open("subset_cnn_dailymail.jsonl", "r") as f:
#     ds_list_loaded = [json.loads(line) for line in f]

# print(f"Loaded {len(ds_list_loaded)} rows.")


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-large")

In [ ]:
# Tokenize the inputs and labels
def preprocess_function(examples):
    inputs = examples['article']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    # Set up the decoder input for PEGASUS
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['factual_summary'], max_length=128, truncation=True)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Fine-tune the model using Trainer API
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=3e-5,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
)

trainer.train()
